### part_A
* separate graph loader for dimacs col format: `loadcol`
* the data is from https://mat.gsia.cmu.edu/COLOR/instances.html 
  (Michael Trick's page)
* dimacs is an old/simple quasy file format described [here](https://mat.gsia.cmu.edu/COLOR/general/ccformat.ps) or [here](ccformat.ps)

In [2]:
import Pkg; Pkg.activate(".")
using Graphs, GraphPlot

  Activating project at `~/Asztal/git/notebooks/graphcol_bt`


In [3]:
function loadcol(gname::String)
  _e(msg)=error("loadcol: $(msg)")
  !isfile(gname) && e("no such file")
  !endswith(gname,".col") && _e("wrong extension")
  gstring=split(read(gname,String),'\n',keepempty=false)
  E=[]
  nV,nE,tV=-1,-1,-1
  for line in gstring
    startswith(line,'c') && continue
    if startswith(line,'p') # only the last counts
      nV,nE=parse.(Int,split(line)[3:end])
      continue
    end
    if startswith(line,'e') 
      a,b=parse.(Int,split(line)[2:end])
      push!(E,(a,b))
      tV=max(tV,max(a,b))
      continue
    end
  end
  if nV<0 || nE<0 || nV!=tV || length(E)!=nE
    _e("wrong data")
  end
  G=Graph()
  add_vertices!(G,nV)
  for (a,b) in E
    add_edge!(G,a,b)
  end
  G
end

# function loadgraph(gname::String; format="lg")
#   if format=="lg"
#     loadgraph(gname)
#   elseif format=="col"
#     loadcol(gname)
#   else
#     error("loadgraph: unknown format")
#   end
# end

loadcol (generic function with 1 method)

#### perform some tests
* how effective is `Graphs.random_greedy_color`?
* on Trick's site the optimal number of colors are also reported (except for some cases): `mincol`

In [4]:
G=loadcol("col-instances/myciel3.col")
println(nv(G)," ",ne(G))
gplot(G,layout=circular_layout)
@time the_coloring=Graphs.random_greedy_color(G,1000)
# mincol=4 

11 20
  0.112528 seconds (88.19 k allocations: 5.902 MiB, 99.14% compilation time)


Graphs.Coloring{Int64}(4, [2, 1, 2, 4, 3, 2, 1, 2, 1, 1, 3])

In [5]:
# this file does not conform the specification -> 
# each edge appears twice
G=loadcol("col-instances/queen5_5.col")
println(nv(G)," ",ne(G))
gplot(G,layout=circular_layout)
@time the_coloring=Graphs.random_greedy_color(G,1000)
# mincol=5

25 160
  0.003870 seconds (27.00 k allocations: 2.228 MiB)


Graphs.Coloring{Int64}(5, [1, 3, 2, 4, 5, 2, 4, 5, 1, 3  …  3, 2, 4, 5, 1, 4, 5, 1, 3, 2])

In [6]:
G=loadcol("col-instances/miles750.col")
println(nv(G)," ",ne(G))
gplot(G,layout=circular_layout)
@time the_coloring=Graphs.random_greedy_color(G,1000)
# mincol=31

128 2113
  0.043442 seconds (130.00 k allocations: 24.659 MiB, 25.28% gc time)


Graphs.Coloring{Int64}(32, [2, 6, 11, 1, 4, 4, 18, 24, 5, 7  …  3, 11, 5, 11, 4, 12, 1, 17, 18, 25])

In [7]:
G=loadcol("col-instances/fpsol2.i.1.col")
println(nv(G)," ",ne(G))
gplot(G,layout=circular_layout)
@time the_coloring=Graphs.random_greedy_color(G,1000)
# mincol=65

496 11654
  0.196557 seconds (498.00 k allocations: 276.920 MiB, 14.32% gc time)


Graphs.Coloring{Int64}(65, [17, 24, 10, 11, 20, 5, 15, 12, 19, 22  …  1, 1, 1, 12, 1, 1, 1, 1, 1, 7])

#### conclusion
* for the inspected instances `Graphs.random_greedy_color` is very 
  effective
* do we need an exact solver at all? :-)